In [2]:
import dask
from dask.distributed import Client, LocalCluster
import os

scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_multinode_dask.json")

dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status"


client = Client(scheduler_file=scheduler_file)
client

<Client: 'tcp://10.249.43.220:8786' processes=12 threads=3072, memory=5.58 TiB>

In [3]:
import xarray as xr
import numpy as np
from datetime import datetime
from calendar import monthrange

In [9]:
def _get_sfc_path_pm(path: str, time, var: str) -> str:
    SFC_VAR_TO_CODE_MAP = {
        "VAR_10U": "128_165_10u",
        "VAR_10V": "128_166_10v",
        "sp": "128_134_sp",
        "tcwv": "128_137_tcwv",
        "VAR_2T": "128_167_2t",
        "msl": "128_151_msl",
        "VAR_100U": "228_246_100u",
        "VAR_100V": "228_247_100v",
    }

    days = monthrange(time.year, time.month)[1]
    code = SFC_VAR_TO_CODE_MAP[var]
    return "{}/{}{:02}/e5.oper.an.sfc.{}.ll025sc.{}{:02}0100_{}{:02}{:02}23.nc".format(
        path,
        time.year,
        time.month,
        code,
        time.year,
        time.month,
        time.year,
        time.month,
        days,
    )

def _get_pressure_path_pm(path: str, time, var: str) -> str:
    PRES_VAR_TO_CODE_MAP = {
        "u": "128_131_u.ll025uv",
        "v": "128_132_v.ll025uv",
        "z": "128_129_z.ll025sc",
        "t": "128_130_t.ll025sc",
        "r": "128_157_r.ll025sc",
    }
    code = PRES_VAR_TO_CODE_MAP[var]
    return "{}/{}{:02}/e5.oper.an.pl.{}.{}{:02}{:02}00_{}{:02}{:02}23.nc".format(
        path,
        time.year,
        time.month,
        code,
        time.year,
        time.month,
        time.day,
        time.year,
        time.month,
        time.day,
    )

In [10]:
M3522 = "/global/cfs/cdirs/m3522/cmip6/ERA5/"

def open_t850(month, hour):
    paths = []
    base_path = M3522 + "/e5.oper.an.pl/"
    for year in range(1993,2017):
        for day in range(1, monthrange(year, month)[1]+1):
            paths.append(_get_pressure_path_pm(base_path, datetime(year, month, day), 't'))
    ds = xr.open_mfdataset(paths).sel(level=850, drop=True)
    return ds.sel(time=ds['time.hour'] == hour)

def open_t2m(month, hour):
    paths = []
    base_path = M3522 + "/e5.oper.an.sfc/"
    for year in range(1993,2017):
        paths.append(_get_sfc_path_pm(base_path, datetime(year, month, 1), 'VAR_2T'))
    ds = xr.open_mfdataset(paths)
    return ds.sel(time=ds['time.hour'] == hour)

In [ ]:
for month in range(1,13):
    for hour in [0,6,12,18]:
        t2m = open_t2m(month, hour)
        t2m = t2m.chunk({'time' : -1, 'latitude' : 20, 'longitude' : 40})
        t2m.mean('time').to_zarr("/pscratch/sd/a/amahesh/hens/thresholds/t2m_mean_{:02d}_{:02d}".format(month,hour), mode='w')
        t2m.std('time').to_zarr("/pscratch/sd/a/amahesh/hens/thresholds/t2m_std_{:02d}_{:02d}".format(month,hour), mode='w')
        t2m.quantile(0.95, dim='time').to_zarr("/pscratch/sd/a/amahesh/hens/thresholds/t2m_percentile95_{:02d}_{:02d}".format(month,hour), mode='w')
        t2m.quantile(0.99, dim='time').to_zarr("/pscratch/sd/a/amahesh/hens/thresholds/t2m_percentile99_{:02d}_{:02d}".format(month,hour), mode='w')
        t2m.quantile(0.999, dim='time').to_zarr("/pscratch/sd/a/amahesh/hens/thresholds/t2m_percentile99p9_{:02d}_{:02d}".format(month,hour), mode='w')
        t2m.max(dim='time').to_zarr("/pscratch/sd/a/amahesh/hens/thresholds/t2m_max_{:02d}_{:02d}".format(month,hour), mode='w')
        
        print("completed {:02d} {:02d}".format(month, hour))

/opt/miniconda3/lib/python3.9/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


completed 07 00
completed 07 06
completed 07 12
completed 07 18
